In [8]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import torch
import edge_tts
import nest_asyncio
from transformers import pipeline

nest_asyncio.apply()

SAMPLE_RATE = 16000
AUDIO_FILE = "recorded.wav"
TTS_FILE = "tts_output.wav"
VOICE = "vi-VN-HoaiMyNeural"

device = 0 if torch.cuda.is_available() else -1

print("🔄 Loading PhoWhisper...")
asr = pipeline(
    "automatic-speech-recognition",
    model="vinai/PhoWhisper-small",
    device=device
)

print("\n🎤 Nói và nhấn ENTER để dừng ghi âm")
audio_chunks = []

def callback(indata, frames, time, status):
    audio_chunks.append(indata.copy())

with sd.InputStream(
    samplerate=SAMPLE_RATE,
    channels=1,
    dtype="float32",
    callback=callback
):
    input()

audio = np.concatenate(audio_chunks, axis=0)
sf.write(AUDIO_FILE, audio, SAMPLE_RATE)

print("\n🧠 Đang nhận dạng...")
result = asr(AUDIO_FILE)
text = result["text"]

print("\n📝 TEXT:")
print(text)

async def tts(text):
    communicate = edge_tts.Communicate(text, VOICE)
    await communicate.save(TTS_FILE)

print("\n🔊 Tổng hợp giọng nói...")
await tts(text)

print("▶️ Phát âm thanh...")
audio_tts, sr = sf.read(TTS_FILE)
sd.play(audio_tts, sr)
sd.wait()

print("✅ Hoàn thành")


🔄 Loading PhoWhisper...


Device set to use cpu



🎤 Nói và nhấn ENTER để dừng ghi âm

🧠 Đang nhận dạng...

📝 TEXT:
tôi muốn tìm hiểu về thông tin của trường đại học fpt cần thơ.

🔊 Tổng hợp giọng nói...
▶️ Phát âm thanh...
✅ Hoàn thành
